In [1]:
#Finger detector

In [2]:
#importing libraries
import cv2
import math
import mediapipe as mp

In [3]:
#required variables
thumb="thumb"
index="index"
middle="middle"
ring="ring"
pinky="pinky"

fingers=[]

wrist_x, wrist_y=0, 0

thumb_tip_x, thumb_tip_y=0, 0
thumb_ip_x, thumb_ip_y=0, 0

index_tip_x, index_tip_y=0, 0
index_pip_x, index_pip_y=0, 0

middle_tip_x, middle_tip_y=0, 0
middle_pip_x, middle_pip_y=0, 0

ring_tip_x, ring_tip_y=0, 0
ring_pip_x, ring_pip_y=0, 0

pinky_tip_x, pinky_tip_y=0, 0
pinky_pip_x, pinky_pip_y=0, 0

global cnt
cnt=0

In [4]:
#finger detecting function
def fingerDetect(tip_x, tip_y, pip_x, pip_y, finger):
    if math.dist([tip_x, tip_y], [wrist_x, wrist_y])>math.dist([pip_x, pip_y], [wrist_x, wrist_y]):
        global cnt
        cnt=cnt+1
        fingers.append(finger)

In [5]:
#capturing the video
cap=cv2.VideoCapture(0)

#detecting the landmarksq
mphand=mp.solutions.hands
hands=mphand.Hands(max_num_hands=1)
mpDraw=mp.solutions.drawing_utils

while True:
    success, img=cap.read()
    img=cv2.flip(img,1) 
    
    #detecting hand/s using 21 points of reference, these reference points will be plotted on the hand
    result=hands.process(img)
    
    if result.multi_hand_landmarks:
        for handlms in result.multi_hand_landmarks:    #loop: handlms gives us the number of hands i.e. 1 hand, 2hands...
            for id, lm in enumerate(handlms.landmark):    #loop: gives us individual reference point on each hand
                cnt=0
                h, w, c=img.shape                         #gives us the dimensions of the frame
                cx, cy=int(lm.x*w), int(lm.y*h)           #give us the coordinates for the reference points
                mpDraw.draw_landmarks(img, handlms, mphand.HAND_CONNECTIONS)
                if id==0:
                    wrist_x, wrist_y=cx, cy
                elif id==4:
                    thumb_tip_x, thumb_tip_y=cx, cy
                elif id==3:
                    thumb_ip_x, thumb_ip_y=cx, cy
                elif id==8:
                    index_tip_x, index_tip_y=cx, cy
                elif id==6:
                    index_pip_x, index_pip_y=cx, cy
                elif id==12:
                    middle_tip_x, middle_tip_y=cx, cy
                elif id==10:
                    middle_pip_x, middle_pip_y=cx, cy
                elif id==16:
                    ring_tip_x, ring_tip_y=cx, cy
                elif id==14:
                    ring_pip_x, ring_pip_y=cx, cy
                elif id==20:
                    pinky_tip_x, pinky_tip_y=cx, cy
                elif id==18:
                    pinky_pip_x, pinky_pip_y=cx, cy
                fingerDetect(thumb_tip_x, thumb_tip_y, thumb_ip_x, thumb_ip_y, thumb)
                fingerDetect(index_tip_x, index_tip_y, index_pip_x, index_pip_y, index)
                fingerDetect(middle_tip_x, middle_tip_y, middle_pip_x, middle_pip_y, middle)
                fingerDetect(ring_tip_x, ring_tip_y, ring_pip_x, ring_pip_y, ring)
                fingerDetect(pinky_tip_x, pinky_tip_y, pinky_pip_x, pinky_pip_y, pinky)
                j=60
                for i in range(0, len(fingers)):
                     cv2.putText(img, fingers[i], (20, 40+j), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
                     j+=40
                fingers.clear()
       
    cv2.putText(img, "Upright fingers: "+str(cnt), (20, 40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
    cv2.imshow("camera", img)
    cnt=0
    if cv2.waitKey(1) & 0xFF==ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break